<center><h1>Facies classification using Machine Learning</h1><center>

<center>
<h2>ICPE 689– Data-Driven Workflows for Intelligent Oil Field Operations and Digital Twins Technology   – Spring 2023</h2>    
<h3>Texas A&amp;M University<br>
Harold Vance Department of Petroleum Engineering<br><br>
Dr. Eduardo Gildin<br>
Marcelo Dall'Aqua<br><br>
 
</h3>
</center>   

# Facies classification using Machine Learning

#### Created by Dr. Eduardo Gildin and Marcelo Dall'Aqua, Harold Vance Department of Petroleum Engineering, Texas A&M

##### Based on the Hall, Brandon - Facies classification using machine learning (2016). Geophysical Tutorial


This notebook demonstrates how to train a machine learning algorithm to predict facies from well log data. The dataset we will use comes from a class excercise from The University of Kansas on [Neural Networks and Fuzzy Systems](http://www.people.ku.edu/~gbohling/EECS833/).  This exercise is based on a consortium project to use machine learning techniques to create a reservoir model of the largest gas fields in North America, the Hugoton and Panoma Fields. For more info on the origin of the data, see [Bohling and Dubois (2003)](http://www.kgs.ku.edu/PRS/publication/2003/ofr2003-50.pdf) and [Dubois et al. (2007)](http://dx.doi.org/10.1016/j.cageo.2006.08.011). 

The dataset we will use is log data from nine wells that have been labeled with a facies type based on oberservation of core.  <font color='red'>We will use this log data to train a several machine learning algorithms to classify facies types. We will use Logistic Regression,  Support vector machines (or SVMs) among others that  are a type of supervised learning model that can be trained on data to perform classification and regression tasks.  The SVM algorithm uses the training data to fit an optimal hyperplane between the different classes (or facies, in our case).  We will use the machine learning  implementation in [scikit-learn](http://scikit-learn.org/stable/modules/svm.html).</font>

First we will [explore the dataset](#Exploring-the-dataset).  We will load the training data from 9 wells, and take a look at what we have to work with.  We will plot the data from a couple wells, and create cross plots to look at the variation within the data.  

Next we will [condition the data set](#Conditioning-the-data-set).  We will remove the entries that have incomplete data.  The data will be scaled to have zero mean and unit variance.  We will also split the data into training and test sets.

We will then be ready to [build the LR classifier].  Later, we will demonstrate how to use the cross validation set to do [model parameter selection](#Model-parameter-selection).

Finally, once we have a built and tuned the classifier, we can [apply the trained model](#Applying-the-classification-model-to-new-data) to classify facies in wells which do not already have labels.  We will apply the classifier to two wells, but in principle you could apply the classifier to any number of wells that had the same log data.

##  $\color{red}{\text{Exploring the dataset (EDA - Exploratory Data Analysis) }}$

First, we will examine the data set we will use to train the classifier.  The training data is contained in the file `facies_vectors.csv`.  The dataset consists of 5 wireline log measurements, two indicator variables and a facies label at half foot intervals.  In machine learning terminology, each log measurement is a **feature vector** that maps a set of **'features'** (the log measurements) to a class (the facies type).  We will use the pandas library to load the data into a dataframe, which provides a convenient data structure to work with well log data.

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pandas import set_option
set_option("display.max_rows", 10)

## Eventually wants to save files (picked models) to a Google drive or harddrive --> follows these steps

In [ ]:
# First thing, need to  google drive to your Colab session.
# You may be asked to sign in or add a code
#from google.colab import drive
#drive.mount('/content/egildin')

In [ ]:
#Then you can simply write to google drive as you would to a local file system like so:

#with open('/content/gdrive/My Drive/file.txt', 'w') as f:
#  f.write('content')

In [ ]:
# Now once done, can download the file.. 
#from google.colab import files
#files.download( "data/dm.ckpt.meta" )

## Pickled a model 

In [ ]:
import joblib

In [ ]:
filename = 'facies_training_data.csv'
training_data = pd.read_csv(filename)
training_data

This data is from the Council Grove gas reservoir in Southwest Kansas.  The Panoma Council Grove Field is predominantly a carbonate gas reservoir encompassing 2700 square miles in Southwestern Kansas.  This dataset is from nine wells (with 4149 examples), consisting of a set of seven predictor variables and a rock facies (class) for each example vector and validation (test) data (830 examples from two wells) having the same seven predictor variables in the feature vector.  Facies are based on examination of cores from nine wells taken vertically at half-foot intervals. Predictor variables include five from wireline log measurements and two geologic constraining variables that are derived from geologic knowledge. These are essentially continuous variables sampled at a half-foot sample rate. 

The seven predictor variables are:
* Five wire line log curves include [gamma ray](http://petrowiki.org/Gamma_ray_logs) (GR), [resistivity logging](http://petrowiki.org/Resistivity_and_spontaneous_%28SP%29_logging) (ILD_log10), [neutron-density porosity difference and average neutron-densityporosity](http://petrowiki.org/Neutron_porosity_logs) (DeltaPHI and PHIND), [photoelectric effect](http://www.glossary.oilfield.slb.com/en/Terms/p/photoelectric_effect.aspx) (PE). Note, some wells do not have PE.
* Two geologic constraining variables: nonmarine-marine indicator (NM_M) and relative position (RELPOS)

The nine discrete facies (classes of rocks) are: 
1. Nonmarine sandstone - SS
2. Nonmarine coarse siltstone - CSiS
3. Nonmarine fine siltstone - FSiS
4. Marine siltstone and shale - SiSh
5. Mudstone (limestone) - MS
6. Wackestone (limestone) - WS
7. Dolomite - D
8. Packstone-grainstone (limestone) - PS
9. Phylloid-algal bafflestone (limestone) - BS

These facies aren't discrete, and gradually blend into one another. Some have neighboring facies that are rather close.  Mislabeling within these neighboring facies can be expected to occur.  The following table lists the facies, their abbreviated labels and their approximate neighbors.

Facies |Label| Adjacent Facies
:---: | :---: |:--:
1 |SS| 2
2 |CSiS| 1,3
3 |FSiS| 2
4 |SiSh| 5
5 |MS| 4,6
6 |WS| 5,7
7 |D| 6,8
8 |PS| 6,7,9
9 |BS| 7,8

Let's clean up this dataset.  The 'Well Name' and 'Formation' columns can be turned into a categorical data type.  

In [ ]:
training_data['Well Name'] = training_data['Well Name'].astype('category')
training_data['Formation'] = training_data['Formation'].astype('category')
training_data['Well Name'].unique()

These are the names of the 10 training wells in the Council Grove reservoir.  Data has been recruited into pseudo-well 'Recruit F9' to better represent facies 9, the Phylloid-algal bafflestone. 

Before we plot the well data, let's define a color map so the facies are represented by consistent color in all the plots in this tutorial.  We also create the abbreviated facies labels, and add those to the `facies_vectors` dataframe.

In [ ]:
# 1=sandstone  2=c_siltstone   3=f_siltstone 
# 4=marine_silt_shale 5=mudstone 6=wackestone 7=dolomite
# 8=packstone 9=bafflestone
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
       '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

facies_labels = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS',
                 'WS', 'D','PS', 'BS']


#facies_color_map is a dictionary that maps facies labels
#to their respective colors
facies_color_map = dict(zip(facies_labels, facies_colors))

## Include a column with the Facies labes                                        
facies_labels_dic = dict(enumerate(facies_labels,start=1))

training_data['FaciesLabels'] = training_data.Facies.map(facies_labels_dic)
training_data.head()

Let's take a quick view of the statistical distribution of the input variables.  

In [ ]:
training_data.describe()

Looking at the `count` values, most values have 4149 valid values except for `PE`, which has 3232.  In this tutorial we will drop the feature vectors that don't have a valid `PE` entry.

In [ ]:
PE_mask = training_data['PE'].notnull().values
training_data = training_data[PE_mask]

Let's take a look at the data from individual wells in a more familiar log plot form.  We will create plots for the five well log variables, as well as a log for facies labels.  The plots are based on the those described in [Alessandro Amato del Monte's tutorial](https://github.com/seg/tutorials/tree/master/1504_Seismic_petrophysics_1).

In [ ]:
def make_facies_log_plot(logs, facies_colors):
    n_facies  = len(facies_colors)
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')

    ztop=logs.Depth.min(); zbot=logs.Depth.max()
    
    cluster=np.repeat(np.expand_dims(logs['Facies'].values,1), 100, 1)
    
    f, ax = plt.subplots(nrows=1, ncols=6, figsize=(8, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[4].plot(logs.PE, logs.Depth, '-', color='black')
    im=ax[5].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=0.5,vmax=n_facies+.5)
    divider = make_axes_locatable(ax[5])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax,ticks=np.arange(1,n_facies+1))
    cbar.set_ticklabels(facies_labels)
  
    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("PHIND")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())
    ax[4].set_xlabel("PE")
    ax[4].set_xlim(logs.PE.min(),logs.PE.max())
    ax[5].set_xlabel('Facies')
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]);
    ax[5].set_yticklabels([]); ax[5].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)

Placing the log plotting code in a function will make it easy to plot the logs from multiples wells, and can be reused later to view the results when we apply the facies classification model to other wells.  The function was written to take a list of colors and facies labels as parameters.  

We then show log plots for well `SHRIMPLIN`

In [ ]:
make_facies_log_plot(
    training_data[training_data['Well Name'] == 'SHRIMPLIN'],
    facies_colors)

and well `SHANKLE`

In [ ]:
make_facies_log_plot(
    training_data[training_data['Well Name'] == 'SHANKLE'],
    facies_colors)

In addition to individual wells, we can look at how the various facies are represented by the entire training set.  Let's plot a histogram of the number of training examples for each facies class.

In [ ]:
#count the number of unique entries for each facies, sort them by
#facies number (instead of by number of entries)
facies_counts = training_data['Facies'].value_counts().sort_index()

#use facies labels to index each count
facies_counts.index = facies_labels

f, ax = plt.subplots(nrows=1, ncols=2, figsize = (20,10))
facies_counts.plot(ax = ax[0], kind='bar',color=facies_colors, fontsize = 20,
                   title='Distribution of Training Data by Facies')

facies_counts.plot.pie(ax = ax[1], colors=facies_colors, fontsize = 20)
facies_counts

In [ ]:
facies_counts = training_data.groupby(['Facies', 'Well Name'])['Facies'].count().unstack()
facies_counts.plot(kind='bar', stacked=True, figsize = (20,10),fontsize = 20)

# Set Axis Properties
plt.xticks(np.arange(9),labels=facies_labels,fontsize = 20)
plt.yticks(fontsize = 15)
plt.title('Distribution of Training Data by Facies Per Well', fontsize = 30)
plt.legend(loc='best', prop={'size': 18})
plt.xlabel('Facies', fontsize = 20)
plt.ylabel('Numbers', fontsize = 20)

This shows the distribution of examples by facies for the 3232 training examples in the training set.  Dolomite (facies 7) has the fewest with 141 examples.  There are also only 185 bafflestone examples.  Depending on the performance of the classifier we are going to train, we may consider getting more examples of these facies.

Crossplots are a familiar tool in the geosciences to visualize how two properties vary with rock type.  This dataset contains 5 log variables, and scatter matrix can help to quickly visualize the variation between the all the variables in the dataset.  We can employ the very useful [Seaborn library](https://stanford.edu/~mwaskom/software/seaborn/) to quickly create a nice looking scatter matrix. Each pane in the plot shows the relationship between two of the variables on the x and y axis, with each point is colored according to its facies.  The same colormap is used to represent the 9 facies.  

In [ ]:
#save plot display settings to change back to when done plotting with seaborn
inline_rc = dict(mpl.rcParams)

import seaborn as sns
#sns.set()

training_data_feaures = training_data.drop(['Well Name','Facies','Formation','Depth','NM_M','RELPOS'],axis=1)

sns.pairplot(training_data_feaures, hue='FaciesLabels', palette=facies_color_map,  hue_order=reversed(facies_labels))

#switch back to default matplotlib plot style
mpl.rcParams.update(inline_rc)


Unfortunetaly, it is not clear from these crossplot what relationships exist between the measurements facies.

We can use heat maps to check on colinearitiy

In [ ]:
fig=plt.figure(figsize=(17,8))
sns.heatmap(training_data_feaures.corr(), cmap='coolwarm', annot=True, linewidths=4, linecolor='black')

##  $\color{red}{\text{Implementation of Machine Learning Algorithms  }}$

## Let's start with Logistic Regression

### Conditioning the data set

Now we extract just the feature variables we need to perform the classification.  The predictor variables are the five wireline values and two geologic constraining variables. We also get a vector of the facies labels that correspond to each feature vector.

In [ ]:
correct_facies_labels = training_data['Facies'].values

feature_vectors = training_data.drop(['Formation', 'Well Name', 'Depth','Facies','FaciesLabels'], axis=1)
feature_vectors.describe()

###  Scaling the dataset

Scikit includes a [preprocessing](http://scikit-learn.org/stable/modules/preprocessing.html) module that can 'standardize' the data (giving each variable zero mean and unit variance, also called whitening). Many machine learning algorithms assume features will be standard normally distributed data (ie: Gaussian with zero mean and unit variance). The factors used to standardize the training set must be applied to any subsequent feature set that will be input to the classifier. The StandardScalar class can be fit to the training set, and later used to standardize any training data.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors)

In [ ]:
scaler

In [ ]:
scaled_features

### Splitting the Dataset

Scikit also includes a handy function to randomly split the training data into training and test sets.  The test set contains a small subset of feature vectors that are not used to train the network.  Because we know the true facies labels for these examples, we can compare the results of the classifier to the actual facies and determine the accuracy of the model.  Let's use 20% of the data for the test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
        scaled_features, correct_facies_labels, test_size=0.3, random_state=42)

### Training the Logistic Regression classifier

Now we use the cleaned and conditioned training set to create a facies classifier.  As mentioned above, we will use a type of machine learning model known as a [Logistic Regression](LG).

The LG implementation in [scikit-learn] (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) takes a number of important parameters.  First we create a classifier using the default settings.  

In [ ]:
from sklearn.linear_model import LogisticRegression 

# We can look at all of the parameter setting for LR --> print out the LR object

LogisticRegression() 

_Note_: One of the most important steps in using these classifiers is the right selection of what is called "Hyperparameters". For instance, "C" controls the intensity of the classifier in some ways - regularization parameter. This improves overfitting models 

In [ ]:
lregression = LogisticRegression() 

_Note_: we can see the inheritance of all other objects, attributes and methods associated with the object LogisticRegression   by simply typying "dir" --> this means you can use .fit or .predict with the regressor

In [ ]:
dir(LogisticRegression)

from time import timeNow we can train the classifier using the training set we created above.

In [ ]:
from time import time

start_train  = time()
lregression.fit(X_train,y_train)
end_train = time()

latency  =  round((end_train - start_train)*1000, 2)
print(' Latency: {} ms'.format(latency))

Now that the model has been trained on our data, we can use it to predict the facies of the feature vectors in the test set.  Because we know the true facies labels of the vectors in the test set, we can use the results to evaluate the accuracy of the classifier.

In [ ]:
start_pred  = time()
y_pred_LR = lregression.predict(X_cv)

end_pred= time()

latency  =  round((end_pred - start_pred)*1000, 2)
print(' Latency: {} ms'.format(latency))

In [ ]:
y_pred_LR

We need some metrics to evaluate how good our classifier is doing.  A [confusion matrix](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/) is a table that can be used to describe the performance of a classification model.  [Scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) allows us to easily create a confusion matrix by supplying the actual and predicted facies labels.

The confusion matrix is simply a 2D array.  The entries of confusion matrix `C[i][j]` are equal to the number of observations predicted to have facies `j`, but are known to have facies `i`.  


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (20,10))

# for i, (a, b) in enumerate(zip(alist, blist)):
for i, (title, normalize) in enumerate(titles_options):
    plot_confusion_matrix(lregression, X_cv, y_cv, ax=ax[i],
                                 display_labels=facies_labels,
                                 normalize=normalize,
                                 cmap='Blues')
    ax[i].set_title(title)

The rows of the confusion matrix correspond to the actual facies labels.  The columns correspond to the labels assigned by the classifier.  For example, consider the first row. For the feature vectors in the test set that actually have label `SS`, 23 were correctly indentified as `SS`, 21 were classified as `CSiS` and 2 were classified as `FSiS`.

The entries along the diagonal are the facies that have been correctly classified.  Below we define two functions that will give an overall value for how the algorithm is performing.  The accuracy is defined as the number of correct classifications divided by the total number of classifications.

In [ ]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

print(classification_report(y_cv, y_pred_LR, target_names=target_names))

In [ ]:
def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i,i]
    acc = total_correct/sum(sum(conf))
    return acc

As noted above, the boundaries between the facies classes are not all sharp, and some of them blend into one another.  The error within these 'adjacent facies' can also be calculated.  We define an array to represent the facies adjacent to each other.  For facies label `i`, `adjacent_facies[i]` is an array of the adjacent facies labels.

In [ ]:
adjacent_facies = np.array([[1], [0,2], [1], [4], [3,5], [4,6,7], [5,7], [5,6,8], [6,7]])

def accuracy_adjacent(conf, adjacent_facies):
    nb_classes = conf.shape[0]
    total_correct = 0.
    for i in np.arange(0,nb_classes):
        total_correct += conf[i,i]
        for j in adjacent_facies[i]:
            total_correct += conf[i,j]
    return total_correct / sum(sum(conf))

In [ ]:
conf = confusion_matrix(y_cv, y_pred_LR)
print('Facies classification accuracy = %f' % accuracy(conf))
print('Adjacent facies classification accuracy = %f' % accuracy_adjacent(conf, adjacent_facies))

### Applying the classification model to new data

Now that we have a trained facies classification model we can use it to identify facies in wells that do not have core data.  In this case, we will apply the classifier to two wells, but we could use it on any number of wells for which we have the same set of well logs for input.

This dataset is similar to the training data except it does not have facies labels.  It is loaded into a dataframe called `test_data`.

In [ ]:
well_data = pd.read_csv('facies_test_data.csv')
well_data['Well Name'] = well_data['Well Name'].astype('category')
well_features = well_data.drop(['Formation', 'Well Name', 'Depth', 'Facies'], axis=1)

In [ ]:
well_features

The data needs to be scaled using the same constants we used for the training data.

In [ ]:
X_test = scaler.transform(well_features)
y_test = well_data['Facies'].values

Finally we predict facies labels for the unknown data, and store the results in a `Facies` column of the `test_data` dataframe.

In [ ]:
#predict facies of unclassified data
# record time for prediction 

start_final  =time()  
y_pred = lregression.predict(X_test)
end_final= time()

latency  =  round((end_final - start_final)*1000, 2)
print(' Latency: {} ms'.format(latency))

#print('Latency: {}ms'.format(latency)

well_data['LR Prediction'] = y_pred
well_data

Print latency time for prediction 

In [ ]:
print(' Latency: {} ms'.format(latency))

Let's see how we did with the confusion matrix:

In [ ]:
cv_conf = confusion_matrix(y_test, y_pred)

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

In [ ]:
well_data['Well Name'].unique()

We can use the well log plot to view the classification results along with the well logs.

In [ ]:
def compare_facies_plot(logs, prediction, facies_colors):
    n_facies  = len(facies_colors)
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')

    ztop=logs.Depth.min(); zbot=logs.Depth.max()
    
    cluster1 = np.repeat(np.expand_dims(logs['Facies'].values,1), 100, 1)
    
    cluster2 = np.repeat(np.expand_dims(logs[prediction].values,1), 100, 1)
    
    f, ax = plt.subplots(nrows=1, ncols=7, figsize=(9, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[4].plot(logs.PE, logs.Depth, '-', color='black')
    im1 = ax[5].imshow(cluster1, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=0.5,vmax=n_facies+.5)
    im2 = ax[6].imshow(cluster2, interpolation='none', aspect='auto',
                     cmap=cmap_facies,vmin=0.5,vmax=n_facies+.5)
    
    divider = make_axes_locatable(ax[6])
    cax  = divider.append_axes("right", size="20%", pad=0.05)
    cbar = plt.colorbar(im2, cax=cax,ticks=np.arange(1,n_facies+1))
    cbar.set_ticklabels(facies_labels)
  
    for i in range(len(ax)-2):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("PHIND")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())
    ax[4].set_xlabel("PE")
    ax[4].set_xlim(logs.PE.min(),logs.PE.max())
    ax[5].set_xlabel('Facies')
    ax[6].set_xlabel(prediction)
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]);
    ax[5].set_xticklabels([]); ax[5].set_xticklabels([])
    ax[6].set_yticklabels([]); ax[6].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)

In [ ]:
compare_facies_plot(
    well_data[well_data['Well Name'] == 'STUART'],
    'LR Prediction',
    facies_colors=facies_colors)

compare_facies_plot(
    well_data[well_data['Well Name'] == 'CRAWFORD'],
    'LR Prediction',
    facies_colors=facies_colors)

##  $\color{red}{\text{Logistic Regression - Hyperparameter Optimzation and K-Folding }}$

## Let's start with Logistic Regression

We will use a fucntion in sklearn called GridSearchcv --> which makes an Exhaustive search (usign K-fold) over specified parameter values for an estimator. This means if you input a number of pre-defined hyperparameters. it will subsititute into our model object and perform K-fold. There are a lot of output functionality as we will see.

## Create a print fucntion 
Now, I will create a function just to print some of the outputs

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

## Perform OPT

The classifier so far has been built with the default parameters. However, we may be able to get improved classification results with optimal parameter choices.

We will consider one parameters. The parameter C is a regularization factor, and tells the classifier how much we want to avoid misclassifying training examples. A large value of C will try to correctly classify more examples from the training set, but if C is too large it may 'overfit' the data and fail to generalize when classifying new data. If C is too small then the model will not be good at fitting outliers and will have a large error on the training set.

We will train a series of classifiers with different values for C. Two nested loops are used to train a classifier for every possible combination of values in the ranges specified. The classification accuracy is recorded for each combination of parameter values. The results are shown in a series of plots, so the parameter values that give the best classification accuracy on the test set can be selected.

This process is also known as 'cross validation'. Often a separate 'cross validation' dataset will be created in addition to the training and test sets to do model selection. For this tutorial we will just use the test set to choose model parameters.

### I do not want to mess up with my previous data, so I will copy to another variable

In [ ]:
X_train_OPT = scaled_features.copy()
y_train_OPT = correct_facies_labels.copy()

We can initialize the K_fold by setting some parameters --> we can decide to use it or not! see below

In [ ]:
# instantiate the Kfold CrossValidation
from sklearn.model_selection import KFold, StratifiedKFold
cv_strategy = KFold(n_splits=3, shuffle = True, random_state=125) # creates an instance
cv_strategy

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'C': [0.0001, 0.01, 1, 5, 10, 20, 50, 100, 1000, 5000, 10000]
}

LR_cv = GridSearchCV(lregression , param_grid=parameters, cv=cv_strategy)
#LR_cv = GridSearchCV(lregression , param_grid=parameters, cv=5)
LR_cv.fit(X_train_OPT, y_train_OPT)

print_results(LR_cv)

Check Best Estimator

In [ ]:
LR_cv.best_estimator_

## *Write* out a pickled model --> Logistic Regression




In [ ]:
joblib.dump(LR_cv.best_estimator_, 'LR_model.pkl')

Display top 5 values of hyperparameter

In [ ]:
LR_cv_results = pd.DataFrame(LR_cv.cv_results_)

LR_cv_results.sort_values(by=['rank_test_score']).head()

## Let's predict the facies with the best model

In [ ]:
best_LR_clf = LogisticRegression(**LR_cv.best_params_)        
best_LR_clf.fit(X_train,y_train)

cv_conf = confusion_matrix(y_test, best_LR_clf.predict(X_test))

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

In [ ]:
#predict facies of unclassified data
y_LR_pred = best_LR_clf.predict(X_test)
well_data['Best LR Prediction'] = y_LR_pred
well_data

In [ ]:
compare_facies_plot(
    well_data[well_data['Well Name'] == 'STUART'],
    'Best LR Prediction',
    facies_colors=facies_colors)

compare_facies_plot(
    well_data[well_data['Well Name'] == 'CRAWFORD'],
    'Best LR Prediction',
    facies_colors=facies_colors)

##  $\color{red}{\text{Support Vecrtor Machine Implementation with  Hyperparameter Optimzation and K-Folding }}$

## Let's start with Basic SVM 

In [ ]:
# Two ways to do it:
#from sklearn import svm
#svm_clf = svm.SVC()

from sklearn.svm import SVC

SVC()

In [ ]:
dir(SVC)

Now, we can just call the SVM function and fit the model as we have done with LR

In [ ]:
svm_clf = SVC()

svm_clf.fit(X_train,y_train)


In [ ]:
y_pred_SVM = svm_clf.predict(X_cv)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

print(classification_report(y_cv, y_pred_SVM, target_names=target_names))

Optmization

In [ ]:
# instantiate the Kfold CrossValidation
from sklearn.model_selection import KFold, StratifiedKFold

cv_strategy = KFold(n_splits=3, shuffle = True, random_state=125) # creates an instance

In [ ]:
parameters_SVM = {
    #'kernel': ['linear', 'rbf'],
    'C': [.01, 1, 5, 10, 20, 100],
    'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10]
    #'C': [.01, 1, 5, 10, 20, 50, 100, 1000, 5000, 10000]
}


#SVM_cv = GridSearchCV(svm_clf, param_grid=parameters_SVM, cv=cv_strategy)
SVM_cv = GridSearchCV(svm_clf, param_grid=parameters_SVM, cv=5)
SVM_cv.fit(X_train_OPT, y_train_OPT)

print_results(SVM_cv)

In [ ]:
best_svm_clf = SVC(**SVM_cv.best_params_)      
best_svm_clf.fit(X_train,y_train)

cv_conf = confusion_matrix(y_cv, best_svm_clf.predict(X_cv))

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

In [ ]:
#predict facies of unclassified data
y_pred_SVM_best = best_svm_clf.predict(X_test)
well_data['Best SVM Prediction'] =y_pred_SVM_best
well_data

In [ ]:
compare_facies_plot(
    well_data[well_data['Well Name'] == 'STUART'],
    'Best SVM Prediction',
    facies_colors=facies_colors)

compare_facies_plot(
    well_data[well_data['Well Name'] == 'CRAWFORD'],
    'Best SVM Prediction',
    facies_colors=facies_colors)

## *Write* out a pickled model --> SVM

In [ ]:
joblib.dump(SVM_cv.best_estimator_, 'SVM_model.pkl')

##  $\color{red}{\text{Decision Trees and Random Forest Implementation with  Hyperparameter Optimzation and K-Folding }}$

## Let's start with Basic Decision Tree (DT)


In [ ]:
from sklearn import tree                                                  # import decision tree from scikit-learn
from sklearn.tree import DecisionTreeClassifier 

In [ ]:
DecisionTree= DecisionTreeClassifier()
DecisionTreeClassifier()

In [ ]:
DT_clf = DecisionTreeClassifier()
DT_clf.fit(X_train,y_train)
# clf = DecisionTreeClassifier(criterion = 'entropy', max_depth = 5)




In [ ]:
y_pred_DT = DT_clf.predict(X_cv)
y_pred_DT

In [ ]:
parameters_DT = {
    'max_leaf_nodes': [3, 5, 10, 15, 20, 100],
    'max_depth': [2, 4, 6, 0.1, 1, 10]
}


#DT_cv = GridSearchCV(DT_clf, param_grid=parameters_DT, cv=cv_strategy)
DT_cv = GridSearchCV(DT_clf, param_grid=parameters_DT, cv=5)
DT_cv.fit(X_train_OPT, y_train_OPT)

print_results(DT_cv)

#### Save optimal

In [ ]:
best_DT_clf = DecisionTreeClassifier(**DT_cv.best_params_)      
best_DT_clf.fit(X_train,y_train)

cv_conf = confusion_matrix(y_cv, best_DT_clf.predict(X_cv))

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

In [ ]:
#predict facies of unclassified data
y_pred_DT_best = best_DT_clf.predict(X_test)
well_data['Best DT Prediction'] =y_pred_DT_best
well_data

In [ ]:
compare_facies_plot(
    well_data[well_data['Well Name'] == 'STUART'],
    'Best DT Prediction',
    facies_colors=facies_colors)

compare_facies_plot(
    well_data[well_data['Well Name'] == 'CRAWFORD'],
    'Best DT Prediction',
    facies_colors=facies_colors)

## *Write* out a pickled model --> Decision Tree

In [ ]:
joblib.dump(DT_cv.best_estimator_, 'DT_model.pkl')

## For the Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()

In [ ]:
 RandomForestClassifier()

In [ ]:
# perform cross-validation
# dictionary with keys
param_grid_strat = {'n_estimators': [10, 100, 200, 500],
                   'min_samples_split': [2, 7, 15]}

# what parameters to search depends on the expert knowledge

RF_cv = GridSearchCV(rforest, param_grid=param_grid_strat, cv=cv_strategy)  # estimator

RF_cv.fit(X_train, y_train)
print(RF_cv.best_params_) 

In [ ]:
best_RF_clf = RandomForestClassifier(**RF_cv.best_params_)        
best_RF_clf.fit(X_train,y_train)

cv_conf = confusion_matrix(y_test, best_RF_clf.predict(X_test))

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

#### Prediction



In [ ]:
#predict facies of unclassified data
y_pred_RF_best = best_RF_clf.predict(X_test)
well_data['Best RF Prediction'] =y_pred_RF_best
well_data

In [ ]:
compare_facies_plot(
    well_data[well_data['Well Name'] == 'STUART'],
    'Best RF Prediction',
    facies_colors=facies_colors)

compare_facies_plot(
    well_data[well_data['Well Name'] == 'CRAWFORD'],
    'Best RF Prediction',
    facies_colors=facies_colors)

## *Write* out a pickled model --> Random Forest:

In [ ]:
joblib.dump(RF_cv.best_estimator_, 'RF_model.pkl')

## For Boosting (or Gradient Boosting using RF)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#clf = GradientBoostingClassifier(loss = 'deviance', max_depth=3, n_estimators = 100)
#clf.fit(X_train, y_train)

GradBoosting = GradientBoostingClassifier()
GradientBoostingClassifier()

In [ ]:
parameters_GB = {
    'n_estimators': [5, 50, 250, 500],   #'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9], # 'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1, 10, 100] # 'learning_rate': [0.01, 0.1, 1, 10, 100]
}

GB_cv = GridSearchCV(GradBoosting, param_grid=parameters_GB, cv=cv_strategy)  # estimator
GB_cv.fit(X_train, y_train)
print(RF_cv.best_params_) 

In [ ]:
best_GB_clf = GradientBoostingClassifier(**GB_cv.best_params_)        
best_GB_clf.fit(X_train,y_train)

cv_conf = confusion_matrix(y_test, best_GB_clf.predict(X_test))

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

## *Write* out a pickled model --> Gradient Boosting

In [ ]:
joblib.dump(GB_cv.best_estimator_, 'GB_model.pkl')

## Other type of boosting

In [ ]:
#XGBoost
from xgboost import XGBClassifier

XGBClassifier()


In [ ]:
# XGB = XGBClassifier(All te parameters)

In [ ]:
#ADABoost

from sklearn.ensemble import AdaBoostClassifier
AdaBoostClassifier()


##  $\color{red}{\text{K-Nearest Neighboor (KNN)}
}$

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN_clf = KNeighborsClassifier()
KNeighborsClassifier()

In [ ]:
# perform cross-validation
# dictionary with keys
param_grid_strat ={'n_neighbors': [5, 10, 50, 70],
                   'algorithm': ['ball_tree', 'kd_tree', 'brute']} 
# what parameters to search depends on the expert knowledge

KNN_cv  = GridSearchCV(KNN_clf, param_grid=param_grid_strat, cv=cv_strategy)  # estimator

KNN_cv.fit(X_train, y_train)
print(KNN_cv.best_params_) 

In [ ]:
best_KNN_clf = KNeighborsClassifier(**KNN_cv.best_params_)        
best_KNN_clf.fit(X_train,y_train)

cv_conf = confusion_matrix(y_test, best_KNN_clf.predict(X_test))

print('Optimized facies classification accuracy = %.2f' % accuracy(cv_conf))
print('Optimized adjacent facies classification accuracy = %.2f' % accuracy_adjacent(cv_conf, adjacent_facies))

In [ ]:
joblib.dump(KNN_cv.best_estimator_, 'KNN_model.pkl')

##  $\color{red}{\text{Summary: Compare model results and final model selection}}$

In this section, we will do the following:
1. Evaluate all of our saved models on the validation set
2. Select the best model based on performance on the validation set
3. Evaluate that model on the holdout test set

### Supposed the dataset is in CSV files (if they are not, we will save it first is CSV so we can retrieve from disk

### Read in Models

In [ ]:
models = {}

#for mdl in ['LR', 'SVM', 'NN', 'RF', 'GB']:
for mdl in ['LR', 'SVM', 'DT', 'RF', 'KNN', 'GB']:
    models[mdl] = joblib.load('{}_model.pkl'.format(mdl))

In [ ]:
#Model = joblib.load('LR_model.pkl'.format('LR'))

In [ ]:
models

In [ ]:
models['LR']

In [ ]:
models.items()

### Evaluate models on the validation set


#### Call all the metrics 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
round(accuracy_score(y_test, y_pred_SVM_best), 3)


In [ ]:
round(precision_score(y_test, y_pred_SVM_best,  average='weighted'), 3)

In [ ]:
round(recall_score(y_test, y_pred_SVM_best , average='weighted'), 3)

In [ ]:
round(f1_score(y_test, y_pred_SVM_best , average='weighted'), 3)

In [ ]:
def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred, average='weighted'), 3)
    recall = round(recall_score(labels, pred,  average='weighted'), 3)
    f1 = round(f1_score(labels, pred,  average='weighted'), 3)
    print('{} -- Accuracy: {} / Precision: {} / Recall: {} /f1: {} /  Latency: {}ms'.format(name,
                                                                                   accuracy,
                                                                                   precision,
                                                                                   recall,
                                                                                   f1,
                                                                                   round((end - start)*1000, 1)))

In [ ]:
for name, mdl in models.items():
    evaluate_model(name, mdl, X_cv, y_cv)

### Evaluate best model on test set

In [ ]:
evaluate_model('Random Forest', models['RF'], X_test, y_test)

## References

Amato del Monte, A., 2015. Seismic Petrophysics: Part 1, *The Leading Edge*, 34 (4). [doi:10.1190/tle34040440.1](http://dx.doi.org/10.1190/tle34040440.1)

Bohling, G. C., and M. K. Dubois, 2003. An Integrated Application of Neural Network and Markov Chain Techniques to Prediction of Lithofacies from Well Logs, *KGS Open-File Report* 2003-50, 6 pp. [pdf](http://www.kgs.ku.edu/PRS/publication/2003/ofr2003-50.pdf)

Dubois, M. K., G. C. Bohling, and S. Chakrabarti, 2007, Comparison of four approaches to a rock facies classification problem, *Computers & Geosciences*, 33 (5), 599-617 pp. [doi:10.1016/j.cageo.2006.08.011](http://dx.doi.org/10.1016/j.cageo.2006.08.011)